In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")
raw_datasets

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [20]:
tokens = raw_datasets["train"][0]["tokens"]
ner = raw_datasets["train"][0]["ner_tags"]
ner_feature = raw_datasets["train"].features["ner_tags"]
for token, int_label in zip(tokens, ner):
    label = ner_feature.feature.int2str(int_label)
    print(f"{token}: {label}, {int_label}")

EU: B-ORG, 3
rejects: O, 0
German: B-MISC, 7
call: O, 0
to: O, 0
boycott: O, 0
British: B-MISC, 7
lamb: O, 0
.: O, 0


In [21]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [23]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [24]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [25]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [28]:
tokens = inputs.tokens()
word_ids = inputs.word_ids()
aligned_labels = align_labels_with_tokens(labels, word_ids)
for token, word_id, label in zip(tokens, word_ids, aligned_labels):
    label_str = word_id if word_id is None else ner_feature.feature.int2str(label)
    print(f"{token}: {label_str}, {label}")

[CLS]: None, -100
EU: B-ORG, 3
rejects: O, 0
German: B-MISC, 7
call: O, 0
to: O, 0
boycott: O, 0
British: B-MISC, 7
la: O, 0
##mb: O, 0
.: O, 0
[SEP]: None, -100


In [29]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [30]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [39]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [40]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [49]:
import evaluate

metric = evaluate.load("seqeval")
metric

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [94]:
import numpy as np


id2label = {i: label for i, label in enumerate(raw_datasets["train"].features["ner_tags"].feature.names)}
label2id = {v: k for k, v in id2label.items()}

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[label] for label in label_set if label != -100] for label_set in labels]
    true_predictions = [
        [id2label[pred] for (pred, label) in zip(pred_set, label_set) if label != -100]
        for pred_set, label_set in zip(predictions, true_labels)
    ]

    #  Compute metrics
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [95]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
model.config.num_labels

9

In [97]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [98]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)

In [99]:
# Evaluate the model for baseline
trainer.evaluate()

{'eval_loss': 2.213622570037842,
 'eval_model_preparation_time': 0.0021,
 'eval_precision': 0.004199901178795793,
 'eval_recall': 0.04005385392123864,
 'eval_f1': 0.007602619389873822,
 'eval_accuracy': 0.11582386530876553,
 'eval_runtime': 12.8528,
 'eval_samples_per_second': 252.863,
 'eval_steps_per_second': 31.666}

In [100]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,0.076600,0.061395,0.002100,0.000328,0.000337,0.000332,0.762362
2,0.035900,0.066799,0.002100,0.000166,0.000168,0.000167,0.764732
3,0.021800,0.062898,0.002100,0.000167,0.000168,0.000167,0.764614


/Users/nivvaknin/Desktop/LLM-course/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/nivvaknin/Desktop/LLM-course/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=5268, training_loss=0.06660345882381014, metrics={'train_runtime': 914.2336, 'train_samples_per_second': 46.075, 'train_steps_per_second': 5.762, 'total_flos': 920771584279074.0, 'train_loss': 0.06660345882381014, 'epoch': 3.0})

In [101]:
trainer.evaluate()

/Users/nivvaknin/Desktop/LLM-course/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.06289765983819962,
 'eval_model_preparation_time': 0.0021,
 'eval_precision': 0.00016655562958027982,
 'eval_recall': 0.0001682935038707506,
 'eval_f1': 0.00016742005692281937,
 'eval_accuracy': 0.7646141166774593,
 'eval_runtime': 12.9851,
 'eval_samples_per_second': 250.286,
 'eval_steps_per_second': 31.343,
 'epoch': 3.0}

In [102]:
model.save_pretrained("bert-finetuned-ner")

In [103]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)


Device set to use mps:0


In [118]:
sample = raw_datasets["test"][0]
sentence = "".join(sample["tokens"])
tokens = sample["tokens"]
ner_tags = [id2label[label] for label in sample["ner_tags"]]
print(f"Sentence: {sentence}")
print("Tokens:", tokens)
print("NER Tags:", ner_tags)

Sentence: SOCCER-JAPANGETLUCKYWIN,CHINAINSURPRISEDEFEAT.
Tokens: ['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
NER Tags: ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O']


In [124]:
pred = ner_pipeline(sentence, aggregation_strategy="simple")
pred

[{'entity_group': 'LOC',
  'score': np.float32(0.839286),
  'word': 'JAPANGETLUCKYWIN',
  'start': 7,
  'end': 23},
 {'entity_group': 'LOC',
  'score': np.float32(0.64975274),
  'word': 'CHINAIN',
  'start': 24,
  'end': 31}]